In [ ]:
import beamtest_analysis_helper as helper
from pathlib import Path
import pandas as pd
import numpy as np
from natsort import natsorted
from collections import defaultdict
import hist
import matplotlib.pyplot as plt
import mplhep as hep
import sqlite3
from tqdm import tqdm
hep.style.use('CMS')

In [ ]:
offset_dfs = defaultdict(pd.DataFrame)

path = Path('/media/daq/X9/')
dirs = natsorted(list(path.glob('Run_DESYFeb2024_Run_18_offset*')))

for idir in tqdm(dirs):
    files = natsorted(list(idir.glob('loop*/TDC*.bin')))
    decoder = helper.DecodeBinary(
        firmware_key=0b0001,
        board_id=[0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f],
        file_list=files,
        save_nem = None,
        skip_filler = True,
    )
    df, _ = decoder.decode_files()
    df = df.drop(columns=['bcid', 'l1a_counter', 'ea'])
    offset_dfs[idir.name.split('Run_18_')[1]] = df

    del df

## Apply TDC cuts

In [ ]:
offset_dfs

In [ ]:
offset_dfs_cut_by_TDC = defaultdict(pd.DataFrame)
good_offsets = []
for key, val in offset_dfs.items():
    print(key, val.size)
    if(val.size<1): continue
    good_offsets += [int(key.split('_')[1])]
    df = val
    tdc_cuts = {}
    for idx in [0, 1, 2, 3]:
        # print(df.loc[df['board'] == idx]['cal'].mode()[0])
        # board ID: [CAL LB, CAL UB, TOA LB, TOA UB, TOT LB, TOT UB]
        if idx == 0:
            tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-50, df.loc[df['board'] == idx]['cal'].mode()[0]+50,  100, 500, 0, 600]
        else:
            tdc_cuts[idx] = [df.loc[df['board'] == idx]['cal'].mode()[0]-50, df.loc[df['board'] == idx]['cal'].mode()[0]+50,  0, 1100, 0, 600]
    # break
    filtered_df = helper.tdc_event_selection(df, tdc_cuts_dict=tdc_cuts, select_by_hit=True)
    offset_dfs_cut_by_TDC[key] = filtered_df

    del filtered_df, df

In [ ]:
good_offsets

## Calculate TOT Mean value

In [ ]:
outfile = 'tot_Mean_by_offsets.sqlite'
# offset_dfs_cut_by_TDC offset_dfs
with sqlite3.connect(outfile) as sqlconn:
    for key, val in offset_dfs_cut_by_TDC.items():
        for idx in [0, 1, 2, 3]:
            sum_group = val[val['board'] == idx].groupby(["col", "row"]).agg({'tot':['mean']})
            sum_group.columns = sum_group.columns.droplevel()
            sum_group.reset_index(inplace=True)

            table_mean = sum_group.pivot_table(index='row', columns='col', values='mean')
            table_mean = table_mean.round(3)

            table_mean = table_mean.reindex(pd.Index(np.arange(0,16), name='')).reset_index()
            table_mean = table_mean.reindex(columns=np.arange(0,16))

            name = key+'_board_id'+str(idx)
            name = name.replace("_", "")
            table_mean.to_sql(name, sqlconn, index=False, if_exists='replace')

## Check dataframes are saved correctly

In [ ]:
# con = sqlite3.connect(outfile)
# data = pd.read_sql_query('SELECT name from sqlite_master where type= "table";', con)
# print(data)

## Draw plot

In [ ]:
with sqlite3.connect(outfile) as conn:
    for idx in [0,1,2,3]:
        tot_means = []
        for ioffset in good_offsets:
            table_name = f'offset{ioffset}boardid{idx}'
            df = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)
            tot_means.append(df.iloc[8,8]) # for now, just a single pixel
        fig, ax = plt.subplots(figsize=(13, 13))
        hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
        ax.set_xlabel('Offset', fontsize=20)
        ax.set_ylabel('TOT Mean', fontsize=20)
        ax.tick_params(axis='both', labelsize=20)
        ax.plot(good_offsets, tot_means)
        ax.set_title(f"Board {idx}, Pixel (8,8)", loc="right")
        plt.tight_layout()